## Deliverable 2. Create a Customer Travel Destinations Map.

In [36]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from keychain import Ggl_key

# Configure gmaps API key
gmaps.configure(api_key=Ggl_key)

In [18]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Resources/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.65,115.33,60.55,68,98,5.30,overcast clouds
1,1,Butaritari,KI,3.07,172.79,80.64,82,100,16.33,moderate rain
2,2,Rikitea,PF,-23.12,-134.97,69.69,78,100,9.95,light rain
3,3,Starokostyantyniv,UA,49.76,27.20,64.60,58,19,4.65,few clouds
4,4,Ushuaia,AR,-54.80,-68.30,35.26,51,20,12.66,few clouds


In [19]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Butaritari,KI,3.07,172.79,80.64,82,100,16.33,moderate rain
5,5,Adrar,MR,20.50,-10.07,89.78,22,4,3.22,clear sky
9,9,Coahuayana,MX,18.73,-103.68,77.47,80,100,3.76,overcast clouds
11,11,Atuona,PF,-9.80,-139.03,77.59,73,9,12.62,clear sky
12,12,Oistins,BB,13.07,-59.53,80.69,83,40,13.80,scattered clouds
14,14,Hun,LY,29.13,15.95,77.58,47,0,4.74,clear sky
16,16,Izumo,JP,35.37,132.77,78.69,83,75,5.75,broken clouds
19,19,Cap Malheureux,MU,-19.98,57.61,75.47,59,3,11.99,clear sky
20,20,Hithadhoo,MV,-0.60,73.08,83.59,76,32,13.69,scattered clouds
21,21,Corn Island,NI,12.17,-83.04,80.85,80,5,12.62,clear sky


In [21]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                250
City                   250
Country                250
Lat                    250
Lng                    250
Max Temp               250
Humidity               250
Cloudiness             250
Wind Speed             250
Current Description    250
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how = "all")
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Butaritari,KI,3.07,172.79,80.64,82,100,16.33,moderate rain
5,5,Adrar,MR,20.50,-10.07,89.78,22,4,3.22,clear sky
9,9,Coahuayana,MX,18.73,-103.68,77.47,80,100,3.76,overcast clouds
11,11,Atuona,PF,-9.80,-139.03,77.59,73,9,12.62,clear sky
12,12,Oistins,BB,13.07,-59.53,80.69,83,40,13.80,scattered clouds


In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,80.64,moderate rain,3.07,172.79,
5,Adrar,MR,89.78,clear sky,20.50,-10.07,
9,Coahuayana,MX,77.47,overcast clouds,18.73,-103.68,
11,Atuona,PF,77.59,clear sky,-9.80,-139.03,
12,Oistins,BB,80.69,scattered clouds,13.07,-59.53,
14,Hun,LY,77.58,clear sky,29.13,15.95,
16,Izumo,JP,78.69,broken clouds,35.37,132.77,
19,Cap Malheureux,MU,75.47,clear sky,-19.98,57.61,
20,Hithadhoo,MV,83.59,scattered clouds,-0.60,73.08,
21,Corn Island,NI,80.85,clear sky,12.17,-83.04,


In [31]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": Ggl_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [42]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
clean_hotel_df = hotel_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng", "Hotel Name"]].copy()

clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,80.64,moderate rain,3.07,172.79,Isles Sunset Lodge
9,Coahuayana,MX,77.47,overcast clouds,18.73,-103.68,El rranchito michuacan de ocampos
11,Atuona,PF,77.59,clear sky,-9.80,-139.03,Villa Enata
12,Oistins,BB,80.69,scattered clouds,13.07,-59.53,Butterfly Beach Hotel
14,Hun,LY,77.58,clear sky,29.13,15.95,Alrwasi Hotel


In [43]:
# 8a. Create the output File (CSV)
output_data_file = "Resources/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [44]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [46]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))